# What is COVID-19?<br>

As  stated  by  WHO  Coronavirus  disease  (COVID-19)  is  an  infectious  disease  caused  by  a  newly  discovered  coronavirus. Most  people  infected  with  the  COVID-19  virus  will  experience  mild  to  moderate  respiratory  illness  and  recover  without requiring  special  treatment. Older people,  and  those  with  underlying  medical  problems  like  cardiovascular  disease,  diabetes, chronic  respiratory  disease,  and  cancer  are  more  likely  to  develop serious  illness.  This  project  aims  at  exploring  COVID-19 through  data  analysis  and  projections.

## Symptoms :
1. Trouble breathing
1. Constant pain or pressure in your chest
1. Bluish lips or face
1. Sudden confusion
1. etc

<img src="SARS-CoV-2-Virus-COVID-19.jpg" length='1000' height='500' >

### Be  strong  and  Keep  safe.




In [1]:
import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objects as go
import plotly_express as px

from functools import reduce

from IPython.core.display import display, HTML 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [2]:
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country_df=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
#confirmed_df.head()

In [4]:
#print('death_df=',death_df.shape)
#print('recovered_df=',recovered_df.shape)
#print('confirmed_df=',confirmed_df.shape)
#print('country_df=',country_df.shape)
#country_df.head()

In [5]:
#Finding total count:

confirmed_total = int(country_df['Confirmed'].sum())
deaths_total = int(country_df['Deaths'].sum())
recovered_total = int(country_df['Recovered'].sum())
active_total = int(country_df['Active'].sum())

In [6]:
#display total 
display(HTML("<div style = 'background-color:#D3D3D3; padding: 30px ; border-color: #6E6E6E ; border-width: .4em; border-style: solid'>" +
             "<span style='color:#4169E1; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color:#A52A2A; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color:#228B22; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [7]:
#Data cleaning

def nan_val(df):
    nan_cols={i : df[i].isnull().sum() for i in df.columns if df[i].isnull().any()}
#    print(nan_cols)
nan_val(confirmed_df)
nan_val(death_df)
nan_val(recovered_df)
nan_val(country_df)



In [8]:
#renaming few columns for ease of typing
for i in [recovered_df,confirmed_df,death_df]:
    i.rename({'Province/State':'state'},axis='columns',inplace=True)
    i.rename({'Country/Region': 'country'}, axis='columns',inplace=True)

# we also deduce that People_Tested and People_Hospitalized column in the country_df dataset do not have any value
#Hence we drop all the above columns and rename a few for ease 

country_df.dropna(axis = 1, how ='all', inplace = True)
country_df.rename({'Country_Region': 'country', 'Long_': 'Long'}, axis='columns',inplace=True)

#for i in [recovered_df,confirmed_df,death_df,country_df]:
#    print(i.columns)

In [9]:
# no duplicate column found
# outliers needed even if present as real time data
# UID and ISO3 may be useful so not removing yet


'''converting all column names to lower case for better ease in future'''
def column_lower(df):
    df.columns = [x.lower() for x in df.columns]
#    print(df.columns)  
column_lower(confirmed_df)
column_lower(death_df)
column_lower(recovered_df)
column_lower(country_df)
        
    
#cleaning complete


## World Choropleth Map for Active cases

In [58]:
#Data visualisation 

#'''World map for active cases (country wise map downloaded from official chloropleth configuration)'''
import plotly as py
import plotly.graph_objects as go


def plot_static(factor,title_bar,col,value):
    fig = go.Figure(data=go.Choropleth(
        locations = country_df['iso3'],
        z = country_df[factor],
        text = country_df['country'],
        colorscale = col,
        autocolorscale=False,
        reversescale=False,
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_title =title_bar,
        
    ))
    fig.update_layout(
        title_text = '{val} Case Count'.format(val = value ),
        title_x = 0.5,
        autosize=True,
        width=1000, height=900,
        geo=dict(
            showframe = False,
            showcoastlines = False,
            bgcolor='lightgrey',
            projection_type = 'equirectangular'
        ))
    fig = go.FigureWidget(fig)

    fig.show()
    
plot_static('active','No of ACTIVE cases','blues', value='Active')


## World Choropleth Map for Recovered cases

In [59]:
plot_static('recovered','No of Recovered cases','greens', value='Recovered')


## World Choropleth Map for Deceased

In [60]:
plot_static('deaths','No of cases of Death','reds',value='Death')

## Overall Conditions check

In [14]:
#Numerical data detailing Country wise 
def cond_check(n):
    n = int(n)
    Updated_situation = pd.DataFrame({'Country Name': country_df['country'], 'Number of Confirmed Cases': country_df['confirmed'],
                          'Number of Deaths': country_df['deaths'],'Number of Recoveries' : country_df['recovered'],
                          'Number of Active Cases' : country_df['active'],})
# number of cases per country/region
    return Updated_situation.head(n).style.background_gradient(cmap='Reds')
interact(cond_check, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears

#'''Current Scenario Sum up'''

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

## Sliding Bar plot for countries with highest mortality rate

In [24]:
mortality_countrywise = country_df[['mortality_rate','country','confirmed','deaths']]

mortality_countrywise=(mortality_countrywise).sort_values(by="mortality_rate",ascending=False)
def mortality_bar(n):
    
    fig = px.bar((mortality_countrywise).head(n), x="mortality_rate", y="country",color='country')
    fig.show()

interact(mortality_bar, n=10)

ipywLayout = widgets.Layout(border='solid 2px ')
ipywLayout.display='none'


interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

## Scatter plot for complete view of mortality rate in the world 

In [56]:
#Mortality rate scatter plot for all countries
fig = px.scatter(mortality_countrywise.sort_values(by="country",ascending=True), x="country", y="mortality_rate", color="mortality_rate",
                hover_data=['country','mortality_rate','confirmed','deaths'])
fig.update_layout( autosize=True, width=1000, height=900,
)
fig = go.FigureWidget(fig)

fig.show()

In [17]:
# 1.2 Tidying the data
# Using melt() command in pandas
id_list = confirmed_df.columns.to_list()[:4]
vars_list =confirmed_df.columns.to_list()[4:]
confirmed_tidy = pd.melt(confirmed_df, id_vars=id_list,\
     value_vars=vars_list, var_name='Date', value_name='Confirmed')
deaths_tidy = pd.melt(death_df, id_vars=id_list,\
     value_vars=vars_list, var_name='Date', value_name='Deaths')
recovered_tidy = pd.melt(recovered_df, id_vars=id_list,\
     value_vars=vars_list, var_name='Date', value_name='Recovered')

# 1.3 Merging the three dataframes into one
data_frames = [confirmed_tidy, deaths_tidy, recovered_tidy]          
df_corona = reduce(lambda left, right: pd.merge(left, right, on =\
               id_list+['Date'], how='outer'), data_frames)

# 1.4 Each row should only represent one observation
id_vars = df_corona.columns[:5]
data_type = ['Confirmed', 'Deaths', 'Recovered']
df_corona = pd.melt(df_corona, id_vars=id_vars,
          value_vars=data_type, var_name='type', value_name='Count')

df_corona['Date'] = pd.to_datetime(df_corona['Date'],
            format='%m/%d/%y', )

#df_corona.head()

## Bar plot for 10 countries with highest death cases:

In [51]:
 
death_time_series=df_corona[df_corona['type']=='Deaths'].iloc[:,:]
death_time_series = (death_time_series[death_time_series['Date'].eq("07/16/2020")].sort_values(by="Count",ascending=False).head(10))
fig = px.bar(death_time_series, x="Count", y="country",color='country',title='COVID Death cases till 16/07/2020')
fig.update_layout( autosize=True,width=1000, height=900,)
fig = go.FigureWidget(fig)
fig.show()


## Worst hit countries ( Confirmed Count ) sliding bar


In [19]:
confirmed_time_series=df_corona[df_corona['type']=='Confirmed'].iloc[:,:]
confirmed_time_series = (confirmed_time_series[confirmed_time_series['Date'].eq("07/15/2020")].sort_values(by="Count",ascending=False))
#print(confirmed_time_series.head())
def bubble_chart(n):
    fig = px.scatter(confirmed_time_series.head(n), x="country", y="Count", size="Count", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px ')
ipywLayout.display='none'

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

## Time series plot for every country:


In [20]:
def plot_cases_of_a_country(country):
    for i in enumerate(df_corona):
        if country == 'World' or country == 'world':
            df_new=df_corona
            
        else:    
            df_new=df_corona[df_corona['country'] == country].iloc[:,:]
            
            
    corona_sums = df_new.groupby(['type', 'Date'],\
                     as_index=False).agg({'Count':'sum'})
    fig = px.line(corona_sums, x='Date', y='Count', color='type',hover_data=['Count'],template='plotly_dark',title='COVID Time series :' + country)

    fig.update_layout(legend_orientation="h")
    fig.show()


In [21]:
country_list=df_corona['country'].unique().tolist()
interact(plot_cases_of_a_country, country=country_list);

ipywLayout = widgets.Layout(border='solid 2px ')
ipywLayout.display='none'


interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

### For more information on COVID visit:

* https://www.cdc.gov/coronavirus/2019-ncov/

* https://www.who.int/emergencies/diseases/novel-coronavirus-2019

### Source Code link:
